In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W48`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W49`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W06`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
3410,2021-02-07 13:10:13+00:00,2021-02-07 20:28:11+00:00,Break,None,N4,shift break,None,KL14,None,Sunday Afternoon,Melanie Dua;,3,437.97,2021-06
24927,2021-02-07 19:12:00+00:00,2021-02-07 19:16:54+00:00,Production,0030 of 0061,B4,None,None,KL3,Wed-NI Delivery,Sunday Afternoon,iXUsr_Erwin(7089),4,4.90,2021-06
41219,2021-02-05 07:02:22+00:00,2021-02-05 07:05:53+00:00,Production,0018 of 0064,A4,None,None,KL2,Sun-NI Delivery,Friday Morning,iXUsr_Ushida(8909),2,3.52,2021-06
24924,2021-02-07 15:27:43+00:00,2021-02-07 15:29:46+00:00,Production,0021 of 0031,F4,None,None,KL5,Wed-NI Delivery,Sunday Afternoon,iXUsr_Erwin(7089),4,2.05,2021-06
41147,2021-02-05 07:55:56+00:00,2021-02-05 08:00:03+00:00,Production,0033 of 0064,A4,None,None,KL2,Sun-NI Delivery,Friday Morning,iXUsr_Ushida(8909),2,4.12,2021-06


In [5]:
data_in_use.shape

(62334, 14)